In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

In [5]:
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

'0.18.0'

In [6]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [7]:
peaks = pd.read_csv("/mnt/c/Users/UVictor/Documents/scATAC_Output/all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

array(['chr1_817076_817576', 'chr1_818534_819034', 'chr1_819632_820132',
       ..., 'chrX_155899158_155899658', 'chrX_155949036_155949536',
       'chrX_155966818_155967318'], dtype=object)

In [8]:
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv("/mnt/c/Users/UVictor/Documents/scATAC_Output/cicero_connections.csv", index_col=0)
cicero_connections.head()

,Peak1,Peak2,coaccess
1,chr10_100005677_100006177,chr10_99761266_99761766,0.000109
2,chr10_100005677_100006177,chr10_99775851_99776351,0.061109
3,chr10_100005677_100006177,chr10_99777878_99778378,-0.014770
4,chr10_100005677_100006177,chr10_99779200_99779700,0.129881
5,chr10_100005677_100006177,chr10_99779735_99780235,0.050904


In [9]:
ma.SUPPORTED_REF_GENOME

,species,ref_genome,provider
0,Human,hg38,UCSC
1,Human,hg19,UCSC
2,Mouse,mm39,UCSC
3,Mouse,mm10,UCSC
4,Mouse,mm9,UCSC
5,S.cerevisiae,sacCer2,UCSC
6,S.cerevisiae,sacCer3,UCSC
7,Zebrafish,danRer7,UCSC
8,Zebrafish,danRer10,UCSC
9,Zebrafish,danRer11,UCSC


In [10]:
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="hg38")

que bed peaks: 527254
tss peaks in que: 39399


In [11]:
tss_annotated.tail()

,chr,start,end,gene_short_name,strand
39394,chr5,149549745,149550245,CSNK1A1,-
39395,chr5,149550391,149550891,CSNK1A1,-
39396,chr5,149551196,149551696,CSNK1A1,-
39397,chr20,10673860,10674360,JAG1,-
39398,chr20,10674653,10675153,JAG1,-


In [12]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated,
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(1927610, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100005677_100006177,BLOC1S2,0.055582
1,chr10_100005677_100006177,DNMBP,0.029999
2,chr10_100005677_100006177,DNMBP-AS1,0.003254
3,chr10_100005677_100006177,ERLIN1,0.017167
4,chr10_100005677_100006177,OLMALINC,0.136850


In [13]:
peak_filter = integrated[integrated.coaccess >= 0.8]
peak_filter = peak_filter[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [14]:
print(peak_filter.shape)
peak_filter.head()

(37077, 2)


,peak_id,gene_short_name
0,chr10_100009705_100010205,DNMBP
1,chr10_100010297_100010797,DNMBP
2,chr10_100185807_100186307,ERLIN1
3,chr10_100186405_100186905,ERLIN1
4,chr10_100229354_100229854,CHUK


In [15]:
peak_filter.to_csv("/mnt/c/Users/UVictor/Documents/scATAC_Output/processed_peak_file.csv")